# Adaboost ou XGBoost ?

Deux algorithmes de boosting connus sont Adaboost & XGBoost, voyons voir la performance de chacun de ces algorithmes sur le dataset d'AIRBNB Seattle. Notre but va être de prédire le prix d'un appartement en fonction des caractéristiques qu'on nous a donné. 

1. Importez les librairies usuelles 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


2. Importez le dataset ```listings.csv```

In [2]:
listings = pd.read_csv("https://raw.githubusercontent.com/JedhaBootcamp/FULL_STACK_12_WEEK_PROGRAM/master/S0.4%20--%20R%C3%A9duction%20de%20Dimensions%20%26%20Boosting/S4-3%20--%20Boosting%20-%20Adaboost%20%26%20XGBoost/S4-3B/Dataset/listings.csv?token=AFB53D5MGH4TUS3R5JTWTPC6HPUAI")
listings.head()

HTTPError: ignored

3. On a beaucoup de données dans ce dataset. Affichez toutes les colonnes du dataset

In [ ]:
listings.info()

4. Révisons un peu Seaborn, affichez la distribution des prix dans le dataset 

In [ ]:
listings.price2 = listings.price.apply(lambda x: x[1:])
listings.price2 = listings.price.str.replace(",","").astype(float)
sns.distplot(listings.price2)

In [ ]:
listings.price = listings.price.apply(lambda x: x[1:])
listings.price = listings.price.replace("1,000.00", "1000").astype(float)
sns.distplot(listings.price)

5. Supprimez les outliers pour ne garder que les appartements qui ont un prix inférieur à 400$/nuit 

In [ ]:
avg_priced_dataset = listings[listings.price <= 400]
avg_priced_dataset.describe()

In [ ]:
sns.distplot(avg_priced_dataset.price)

6. Exprimez le prix en fonction des variables suivantes : 

- ```room type```
- ```beds```
- ```property type```

In [ ]:
sns.catplot(x="room_type", y="price", data=avg_priced_dataset, kind="bar")

In [ ]:
sns.catplot(x="beds", y="price", data=avg_priced_dataset, kind="bar")

In [ ]:
avg_priced_dataset[avg_priced_dataset.beds==8]

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
sns.catplot(x="property_type", y="price", data=avg_priced_dataset, kind="bar", ax=ax)

7. Séparez la variable cible de votre dataset

In [ ]:
X = avg_priced_dataset.loc[:, avg_priced_dataset.columns!="price"]
#X = avg_priced_dataset.drop(["price"], axis = 1)
y = avg_priced_dataset.loc[:, avg_priced_dataset.columns=="price"]

8. Il faut qu'on écrème quelques variables explicatives de notre jeu de données. Commencer par simplement enlever les variables qui expriment un ```id``` quelconque ou urls. On enlèvera également les variables qui contiennent des textes longs comme ```notes```

De la même manière, on enlèvera  toutes les variables qui décrivent le prix mensuel ou hebdomadaire comme  ```monthly price```

Votre dataset devrait contenir uniquement les variables catégoriques et numériques une fois votre nettoyage fait. 

A la fin, votre dataset contiendra les variables suivantes : 

```
Index(['experiences_offered', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'bed_type', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 'calculated_host_listings_count',
       'reviews_per_month'],
      dtype='object')
```

In [ ]:
a=X.isnull().sum()/X.shape[0]*100
a.sort_values(ascending=False)

In [ ]:
X_clean = X.drop(["host_since", 
            "host_location", 
            "host_about", 
            "host_thumbnail_url", 
            "host_picture_url",
           "license",
           "monthly_price",
           "square_feet",
            "host_verifications", 
        "street", 
        "neighbourhood", 
        "neighbourhood_cleansed", 
        "city", 
        "state", 
        "market", 
        "smart_location",
        "country_code",
        "country",
        "is_location_exact",
        "amenities",
        "jurisdiction_names",
        "first_review",
        "last_review",
        "calendar_last_scraped",
        "weekly_price",
            "notes",
                  "neighborhood_overview",
         "name",
                  "description",
                  "listing_url" ,
"host_id",
"host_url",
                 "scrape_id",
                 "space",
                 "picture_url",
                 "last_scraped",
                 "host_name",
                 "medium_url",
                 "summary",
                 "xl_picture_url",
                 "transit",
                 "thumbnail_url",
                 "host_neighbourhood",
                 "zipcode",
                 "calendar_updated",
                 "license","id"], axis=1)

9 Reste-t-il des valeurs manquantes? Certaines peuvent elles être traîtées sans avoir recours à l'interpolation ? Les variables sont elles toutes dans un format adapté ? Si non appliquez des transformations adaptées.

In [ ]:
X_clean.isnull().sum().sort_values()

In [ ]:
# Pour la suite, nous remplirons les valeurs NaN de la manière la plus simple possible 
X_clean.cleaning_fee = X_clean.cleaning_fee.apply(lambda x: float(x[1:])/100 if type(x) == str else x)
X_clean.cleaning_fee = X_clean.cleaning_fee.fillna(0)

In [ ]:
# Nous devons enlever le caractère $ de chacun des prix pour rendre les variables numériques
X_clean["host_response_rate_clean"] = X_clean.host_response_rate.apply(lambda x: float(x[:-1])/100 if type(x) == str else x)
X_clean = X_clean.drop("host_response_rate", axis =1)


In [ ]:
X_clean["host_acceptance_rate_clean"] = X_clean.host_acceptance_rate.apply(lambda x: int(x[:-1])/100 if type(x) == str else x)
X_clean = X_clean.drop("host_acceptance_rate", axis = 1)

In [ ]:
X_clean.host_has_profile_pic = X_clean.host_has_profile_pic.fillna("f")
X_clean.host_identity_verified = X_clean.host_identity_verified.fillna("f")

X_clean.host_response_time = X_clean.host_response_time.fillna("Unknown")
X_clean.host_is_superhost = X_clean.host_is_superhost.fillna("Unknown")
X_clean.property_type = X_clean.property_type.fillna("unknown")

X_clean.extra_people = X_clean.extra_people.apply(lambda x: float(x[1:]))
X_clean.security_deposit = X_clean.security_deposit.apply(lambda x: float(str(x).replace(",","").replace("nan","$0")[1:]))
X_clean.security_deposit = X_clean.security_deposit.fillna(0)


10. Vérifiez que toutes les variables numériques le sont effectivement bien. (N'oubliez pas de regarder y)

In [ ]:
y = y.apply(lambda x: float(x[1:]) if type(x)==str else x)

11. Séparez les variables en deux groupes, les variables numériques d'un côté et les qualitatives de l'autre.

In [ ]:
num_col = X_clean.select_dtypes([np.number]).columns
cat_col = X_clean.select_dtypes("object").columns
 
X_num = X_clean[num_col]
X_cat =  pd.get_dummies(X_clean[cat_col],drop_first=True)

12. Faites maintenant un ```train_test_split```

In [ ]:
from sklearn.model_selection import train_test_split



X_train_num, X_test_num, X_train_cat, X_test_cat, y_train, y_test = train_test_split(X_num, X_cat, y, test_size =0.2)

In [ ]:
X_train_cat.shape

13. Normalisez ```X_train_num``` & ```X_test_num``` et rassemblez les deux groupes de variables

In [ ]:
from sklearn.preprocessing import StandardScaler 
sc_x = StandardScaler()


X_train_num_norm = pd.DataFrame(sc_x.fit_transform(X_train_num),columns=X_train_num.columns, index = X_train_num.index)
X_test_num_norm = pd.DataFrame(sc_x.transform(X_test_num),columns=X_test_num.columns, index = X_test_num.index)

X_train = pd.merge(X_train_num_norm, X_train_cat,left_index=True, right_index=True)
X_test = pd.merge(X_test_num_norm, X_test_cat,left_index=True, right_index=True)



14. Gérez les valeurs NaN. Utilisez les stratégies que vous préférez 

In [ ]:
!pip install missingpy
from missingpy import KNNImputer
imputer = KNNImputer()

X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


15. Entrainez d'abord un modèle d'Adaboost standard et regardez votre score 


In [ ]:

from sklearn.ensemble import AdaBoostRegressor 
regressor_ada = AdaBoostRegressor()
regressor_ada.fit(X_train, y_train)
regressor_ada.score(X_test, y_test)

#np.sqrt(np.mean((regressor_ada.predict(X_test) - y_test.values)**2))

In [ ]:
AdaBoostRegressor?

In [ ]:
np.mean(y_test)

16. Entrainez ensuite un modèle XGBoost et regardez votre score 

In [ ]:
!pip install xgboost
from xgboost import XGBRegressor
regressor_xgb = XGBRegressor()
regressor_xgb.fit(X_train, y_train)
regressor_xgb.score(X_test, y_test)

17. Par défaut, Adaboost prend des decision trees comme modèle a booster. Tentez de mettre une regression linéaire 

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
regressor_ada = AdaBoostRegressor(lin_reg)
regressor_ada.fit(X_train, y_train)
regressor_ada.score(X_test, y_test)

In [ ]:
regressor_ada.score(X_train, y_train)


18. La régression linéaire n'était pas la meilleure idée mais peut être qu'on peut faire une grid_search sur le ```learning rate``` & ```n_estimators```pour rattraper le score de XGBoost ?

In [ ]:
from sklearn.model_selection import GridSearchCV
regressor_ada = AdaBoostRegressor()
parameters = {'n_estimators':[40,50,70, 100, 150, 200],
              "learning_rate":[1.0, 0.9, 0.8, 0.7, 0.6, 0.5]}

regressor_ada_best = GridSearchCV(regressor_ada, parameters)
regressor_ada_best.fit(X_train, y_train)

In [ ]:
regressor_ada_best.best_params_

In [ ]:
regressor_ada_best.best_estimator_.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
regressor_ada = AdaBoostRegressor(n_estimators=40,learning_rate=0.5)
regressor_ada.fit(X_train, y_train)

In [ ]:
regressor_ada.score(X_test,y_test)

---> Même avec beaucoup plus d'estimateurs, nous n'arrivons pas au score de XGBoost

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_regressor = DecisionTreeRegressor()
tree_regressor.fit(X_train, y_train)


In [ ]:
tree_regressor.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import BaggingRegressor
bagging_regressor = BaggingRegressor(n_estimators=40)
bagging_regressor.fit(X_train, y_train)

In [ ]:
bagging_regressor.score(X_test,y_test)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(learning_rate=0.5,n_estimators=40)
gbr.fit(X_train, y_train)

In [ ]:
gbr.score(X_test,y_test)

In [ ]:
from sklearn.externals import joblib
joblib.dump(gbr, 'test.joblib') 

In [ ]:
loaded_model = joblib.load('test.joblib')
result = loaded_model.score(X_test,y_test)
print(result)